## Setting up 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim as gs
import slugify as sg
#%qtconsole


Slow version of gensim.models.doc2vec is being used


In [2]:
data = pd.read_table('SMSSpamCollection',header= None, names = ('outcome', 'content'))

### 2. clean up, remove common words and 1 time words

In [3]:
# lower case it
clean = data["content"] = list(data["content"].str.lower())

# set up to remove common words #
words_to_remove = set('for a of the and to in or'.split())

clean = [[word for word in document.split()] for document in clean]



In [4]:
# Extract Unique Characters ( work in progress)
from re import sub # import sub to replace items in the followiong list comprehension
x = [[sub(pattern='[a-z0-9]',repl='', string=w) for w in x] for x in clean] # remove all regular alphanumeric and integers
x = set([item for sublist in x for item in sublist]) # list comp. to flatten
x = set(list(''.join(x))) # first, concatenate all characters, then use list to separate, then set to gain uniques

In [5]:
# remove symbols
clean = [[sub('\\'+'|\\'.join(x),'',word) for word in text] for text in clean]
# remove common words
clean = [[word for word in document if word not in words_to_remove] for document in clean]

In [6]:
# Count Freqs
from collections import defaultdict
frequency = defaultdict(int)
for text in clean:
    for token in text:
        frequency[token] += 1

# get freq > 1
clean = [[x for x in words if frequency[x] > 1] for words in clean]



### 3. Create Dictionary 

In [7]:
dictionary = gs.corpora.Dictionary(clean)
# test
dictionary.doc2bow("'for a of the and to in or".split())



[]

### 4. Create Corpus

In [8]:
corpus = [dictionary.doc2bow(text) for text in clean]

### 5. Create xform

In [9]:
tfidf = gs.models.TfidfModel(corpus) 
corp_tf = tfidf[corpus] 

In [10]:
lsi = gs.models.LsiModel(corp_tf, id2word=dictionary, num_topics = 300)
corp_tf = lsi[corp_tf] 